In [ ]:
from PIL import Image

import torch
from transformers import BeitFeatureExtractor, BeitForImageClassification
import pandas as pd
from torch.utils.data import random_split, DataLoader
import os
from dataset import SingleImageDataset


In [ ]:
dataset = SingleImageDataset()
train_set, val_set, test_set = random_split(dataset, [0.6, 0.2, 0.2])

In [ ]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

In [ ]:
for images, labels in train_loader:
    print(images.size())
    print(labels.size())

In [ ]:
feature_extractor = BeitFeatureExtractor.from_pretrained('microsoft/beit-base-patch16-384')
model = BeitForImageClassification.from_pretrained('microsoft/beit-base-patch16-384')

In [ ]:
image = Image.open('../data/images/0,37.008870562004056,-8.939049125230865/138.jpg')

image

In [ ]:
model.classifier = torch.nn.Linear(768, 2)

In [ ]:
model